<a href="https://colab.research.google.com/github/yoseforaz0990/ML-templates/blob/main/deep_learning/Recurrent%20Neural%20Networks%20(RNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

| Step                                   | Description                                                                              |
|----------------------------------------|------------------------------------------------------------------------------------------|
| 1. Importing Libraries                 | Importing the required libraries, including Pandas, NumPy, and MinMaxScaler from sklearn.|
| 2. Loading Training Dataset           | Reading the training dataset (Google_Stock_Price_Train.csv) using Pandas.                |
| 3. Feature Scaling                    | Scaling the training data using MinMaxScaler to normalize the values between 0 and 1.   |
| 4. Creating Data Structure with Timesteps and Output  | Preparing the training data with a sequence of 60 timesteps and 1 output.                |
| 5. Reshaping Data                     | Reshaping the data to fit the LSTM input shape (samples, timesteps, features).           |
| 6. Importing Keras Libraries and Packages             | Importing necessary Keras libraries, including Sequential, LSTM, Dropout, and Dense.    |
| 7. Initializing the RNN                               | Initializing the RNN model using the Sequential class.                                  |
| 8. Adding LSTM Layers and Dropout Regularization      | Adding four LSTM layers with dropout regularization to the RNN.                          |
| 9. Adding the Output Layer                            | Adding the output layer with one neuron for regression.                                 |
| 10. Compiling the RNN                                  | Compiling the RNN model with the Adam optimizer and mean squared error loss function.   |
| 11. Fitting the RNN to the Training Set                | Fitting the RNN model to the training data with 100 epochs and batch size of 32.       |
| 12. Getting Real Stock Price of 2017                   | Retrieving the real stock prices of 2017 (out of the training set range) for comparison. |
| 13. Getting Predicted Stock Price of 2017              | Making predictions for the stock prices of 2017 using the trained RNN model.           |
| 14. Visualizing the Results                            | Plotting the real and predicted stock prices of 2017 for visualization.                 |


In [ ]:
# Importing the libraries

# Importing the training set
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')
training_set = dataset_train.iloc[:, 1:2].values

# Feature Scaling
sc = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = sc.fit_transform(training_set)

# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []
for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units=1))

# Compiling the RNN
regressor.compile(optimizer='adam', loss='mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs=100, batch_size=32)

# Getting the real stock price of 2017
dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values

# Getting the predicted stock price of 2017
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), axis=0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1, 1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

# Visualising the results
plt.plot(real_stock_price, color='red', label='Real Google Stock Price')
plt.plot(predicted_stock_price, color='blue', label='Predicted Google Stock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()
